In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Our Libs 

In [9]:
import re
import pandas 
import numpy
from pathlib import Path

# List All files 

In [15]:
def list_data_files(): 
    files = []
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files
         
def get_files_regex(file_name_str = "test"): 
    raw_files = list_data_files()
    regex = re.compile(f".+{file_name_str}.+txt")  
    raw_data_files = [f for f in raw_files if re.match(regex, f)]
    return raw_data_files

## Load to dataframe / 读取到 pandas DataFrames 

In [18]:
def read_data_files(file_name_str = "train", use_pd = True, sep = " ", columns = None):  
    if not columns:
        columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
            "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
            ,"sensor20","sensor21" ]  

    raw_data_files = get_files_regex(file_name_str =file_name_str) 

    df_total =  pd.DataFrame() 
    for f in raw_data_files: 
        if use_pd:
            df_ = pd.read_csv(f,  index_col = False, names = columns, sep=' ')
        else: 
            df_= pd.DataFrame(np.loadtxt(f), columns=columns) 
        df_[["id", "cycle"]] = df_[["id", "cycle"]].astype(int)
            
        flag = re.findall(r"FD\d{3}", str(f))[0]
        df_["Flag"] = flag 
        if df_total.empty:
            df_total = df_.copy()
        else: 
            df_total = pd.concat([df_total, df_], axis = 0 ) 
    
    return df_total 

In [21]:
 def read_result(file_name_str = "RUL_FD", use_pd = True, sep = " ", columns = None):
    raw_data_files = get_files_regex(file_name_str =file_name_str)  
    if not columns:
        columns = ["rul"]

    df_result =  pd.DataFrame() 
    for f in raw_data_files:
        if use_pd: 
            df_ = pd.read_csv(f,  index_col = False, names = columns, sep = sep)
        else:
            df_= pd.DataFrame(np.loadtxt(f), columns = columns) 
        flag = re.findall(r"FD\d{3}", str(f))[0]
        df_["Flag"] = flag 
        if df_result.empty:
            df_result = df_.copy()
        else: 
            df_result = pd.concat([df_result, df_], axis = 0 ) 
    return df_result

# Sum Up: prepare train, test and result file / 主程序

In [24]:
 def prepare_dfs(use_pd = True, sep = " "): 
      
     columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
          "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
          ,"sensor20","sensor21" ] 

     # Train
     df_train = read_data_files( file_name_str = "train", use_pd = use_pd, sep = " ", columns = columns)
     # Test
     df_test = read_data_files( file_name_str = "test", use_pd = use_pd, sep = " ", columns = columns)

     resul_columns = ["rul"]
     df_result = read_result(file_name_str = "RUL_FD", \
          use_pd = use_pd, sep =sep, columns = resul_columns)
 
     df_train.iloc[:, [0,1]] = df_train.iloc[:, [0,1]].astype(int)
     df_test.iloc[:, [0,1]] = df_test.iloc[:, [0,1]].astype(int) 

     df_max = df_test.groupby(["Flag","id"])["cycle"].max().reset_index()
     df_result = df_result.reset_index()
     df_result["id"] = df_result.groupby("Flag")["index"].rank("first", ascending = True).astype(int)
     df_result.drop(columns = ["index"], inplace = True)
     
     df_result = df_result.merge(df_max, on = ["Flag", "id"], how = "inner")
      
     df_result["rul_failed"] = df_result["rul"] + df_result["cycle"]

     df_test = df_test.merge(df_result[["rul_failed", "Flag", "id"]], on = ["Flag", "id"], how = "inner")
     df_test["remaining_rul"] = df_test["rul_failed"] - df_test["cycle"]

     #df_test[["rul_failed", "remaining_rul"]] = df_test[["rul_failed", "remaining_rul"]].astype(int)
     return df_train, df_test, df_result